In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split

In [50]:
# Load the dataset
file_path = "data/DeDoDe_Descriptors_Dataset.pth"  # Change this to your actual path
data = torch.load(file_path)
base_descriptors = data['descriptors']  # Assuming these keys exist
transformed_descriptors = data['deformed_descriptors']

/tmp/ipykernel_21061/826197558.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [51]:
# Select the data corresponding to transformation 0
transformations = data['transformations']
transformations = torch.repeat_interleave(transformations, repeats=8, dim=1).flatten()
idx = transformations == 31
base_descriptors = base_descriptors[idx]
transformed_descriptors = transformed_descriptors[idx]

In [52]:
print(base_descriptors)
print(transformed_descriptors[0].mean())
print(torch.nn.functional.cosine_similarity(base_descriptors,transformed_descriptors,dim = 1).mean())


tensor([[ 0.8651,  0.2148,  0.1459,  ..., -0.3268,  0.1482,  0.4894],
        [-0.3635,  0.0501,  0.0675,  ..., -0.6345, -0.5220,  0.2216],
        [-0.0612,  0.5054,  0.1808,  ..., -0.2336, -1.0503, -0.0952],
        ...,
        [-0.5589,  0.1584,  0.0729,  ..., -0.6847,  0.3972,  0.2990],
        [ 0.5017,  0.2304, -0.1690,  ...,  0.5677, -0.1402, -0.5549],
        [ 0.9935, -0.2173, -0.8181,  ...,  1.5885,  0.0631, -0.5063]],
       dtype=torch.float64)
tensor(-0.0361, dtype=torch.float64)
tensor(0.0546, dtype=torch.float64)


In [58]:
# Create a dataset and dataloader
dataset = TensorDataset(base_descriptors, transformed_descriptors)
train_size = 0.8
val_size = 0.2

train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [59]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            # nn.Linear(input_dim,output_dim)
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.model(x)

In [60]:
# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
input_dim = base_descriptors.shape[1]
output_dim = transformed_descriptors.shape[1]
model = MLP(input_dim,output_dim).double().to(device)

cuda


In [61]:
# Define loss function and optimizer
# criterion = nn.CosineEmbeddingLoss()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [62]:
# Training loop
epochs = 250
# cosine_target = torch.ones(8).to(device)
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for base, transformed in train_dataloader:
        base, transformed = base.to(device), transformed.to(device)
        
        optimizer.zero_grad()
        output = model(base)
        loss = criterion(output, transformed)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for base, transformed in val_dataloader:
            base, transformed = base.to(device), transformed.to(device)
            output = model(base)
            loss = criterion(output,transformed)
            val_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(val_dataloader)

    if (epoch+1)%10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

Epoch 10/250, Train Loss: 0.059786, Val Loss: 0.199120
Epoch 20/250, Train Loss: 0.034859, Val Loss: 0.214445
Epoch 30/250, Train Loss: 0.027924, Val Loss: 0.221344
Epoch 40/250, Train Loss: 0.024899, Val Loss: 0.226538
Epoch 50/250, Train Loss: 0.022385, Val Loss: 0.227303
Epoch 60/250, Train Loss: 0.021377, Val Loss: 0.228174
Epoch 70/250, Train Loss: 0.020165, Val Loss: 0.230588
Epoch 80/250, Train Loss: 0.019428, Val Loss: 0.232092
Epoch 90/250, Train Loss: 0.018981, Val Loss: 0.231298
Epoch 100/250, Train Loss: 0.018303, Val Loss: 0.232174
Epoch 110/250, Train Loss: 0.018125, Val Loss: 0.232820
Epoch 120/250, Train Loss: 0.017345, Val Loss: 0.232749
Epoch 130/250, Train Loss: 0.016414, Val Loss: 0.233026
Epoch 140/250, Train Loss: 0.016956, Val Loss: 0.233865
Epoch 150/250, Train Loss: 0.016569, Val Loss: 0.233926
Epoch 160/250, Train Loss: 0.015609, Val Loss: 0.233762
Epoch 170/250, Train Loss: 0.016089, Val Loss: 0.232676
Epoch 180/250, Train Loss: 0.015100, Val Loss: 0.233689
E

In [160]:
# Save the trained model
torch.save(model.state_dict(), "models/single_transformation_model.pth")
print("Model training complete and saved.")

Model training complete and saved.
